In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "yjbds/ml-models:0.4.8"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'yjbds/ml-models:0.4.8'


In [3]:
import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "yjb", "framework": "sklearn"}
fn.export("function.yaml")

[mlrun] 2020-05-26 01:41:47,527 function spec saved to path: function.yaml


In [7]:
from mlutils import get_vol_mount
fn.apply(get_vol_mount())

## tests

In [8]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [9]:
run = fn.run(NewTask(**task_params), artifact_path=mlconf.artifact_path + "/toy-data")

[mlrun] 2020-05-26 01:41:48,192 starting run tasks load toy dataset uid=3ee146a36db3402fb5684a5708e90e49  -> http://mlrun-api:8080
[mlrun] 2020-05-26 01:41:48,292 Job is running in the background, pod: tasks-load-toy-dataset-7mjdw
[mlrun] 2020-05-26 01:41:51,764 starting local run: main.py # load_dataset
[mlrun] 2020-05-26 01:41:52,105 log artifact wine at /User/artifacts/toy-data/wine.parquet, size: 18454, db: Y

[mlrun] 2020-05-26 01:41:52,133 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...08e90e49,0,May 26 01:41:51,completed,tasks load toy dataset,v3io_user=adminkind=jobowner=adminhost=tasks-load-toy-dataset-7mjdw,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3ee146a36db3402fb5684a5708e90e49  , !mlrun logs 3ee146a36db3402fb5684a5708e90e49 
[mlrun] 2020-05-26 01:41:54,490 run executed, status=completed


### or locally

In [10]:
from mlrun import run_local

In [11]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path + "/toy-data")

[mlrun] 2020-05-26 01:41:54,532 starting run mlrun-6c4940-load_dataset uid=87874f84e696493b97c8639cfd0a1442  -> http://mlrun-api:8080
[mlrun] 2020-05-26 01:41:54,744 log artifact wine at /User/artifacts/toy-data/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...fd0a1442,0,May 26 01:41:54,completed,mlrun-6c4940-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f9d6597fd-ns9cj,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 87874f84e696493b97c8639cfd0a1442 --project default , !mlrun logs 87874f84e696493b97c8639cfd0a1442 --project default
[mlrun] 2020-05-26 01:41:54,816 run executed, status=completed
[mlrun] 2020-05-26 01:41:54,822 starting run mlrun-8d3cf4-load_dataset uid=e14697cee74446efa7192e4dee25d761  -> http://mlrun-api:8080
[mlrun] 2020-05-26 01:41:54,914 log artifact iris at /User/artifacts/toy-data/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...ee25d761,0,May 26 01:41:54,completed,mlrun-8d3cf4-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f9d6597fd-ns9cj,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run e14697cee74446efa7192e4dee25d761 --project default , !mlrun logs e14697cee74446efa7192e4dee25d761 --project default
[mlrun] 2020-05-26 01:41:54,956 run executed, status=completed
[mlrun] 2020-05-26 01:41:54,963 starting run mlrun-6b7ee9-load_dataset uid=3a538d97f311478487c9537d3b79f3f6  -> http://mlrun-api:8080
[mlrun] 2020-05-26 01:41:55,181 log artifact breast_cancer at /User/artifacts/toy-data/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...3b79f3f6,0,May 26 01:41:54,completed,mlrun-6b7ee9-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f9d6597fd-ns9cj,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3a538d97f311478487c9537d3b79f3f6 --project default , !mlrun logs 3a538d97f311478487c9537d3b79f3f6 --project default
[mlrun] 2020-05-26 01:41:55,251 run executed, status=completed
